In [2]:
import kquant as kq
CFG = {"sampling_cnt": 150}
# API_account
def set_api_account():
    kq.set_api("KRX2308020", "EQDkUcyI3dK6oIAXqAR8BXOK4bKxHHmH")
    return None

set_api_account()

In [3]:
"""
General

DataLoader
    - STATUS_LOADER
    - SYMBOL_LOADER

"""
# LOADER
import datetime as dt
import pandas as pd
import kquant as kq


class STATUS_LOADER:
    """
    STATUS_LOADER : 상태 정보 추출 클래스

    Methods:
        - __init__
        - get_current_cash
        - get_status_df
    """

    def __init__(self, dict_df_result, dict_df_position) -> None:
        """
        STATUS_LOADER의 생성자

        Args:
            dict_df_result (dict(str,pd.DataFrame))
            dict_df_position (dict(str,pd.DataFrame))

        """
        self.dict_df_result = dict_df_result
        self.dict_df_position = dict_df_position

    def get_current_cash(self) -> float:
        """
        현재 보유 현금을 반환하는 메서드
            - 만약 dict_df_result에서 CASH column을 찾을 수 없다면 초기투자금인 10억을 반환합니다.

        Returns:
            float: 현재 보유 현금
        """
        _dict_df_result = self.dict_df_result
        try:
            _df_result_total = _dict_df_result["TOTAL"]
            _current_cash = (
                _df_result_total.sort_values("DATE").tail(1)["CASH"].values[0]
            )
            return _current_cash
        except:
            return 1_000_000_000.0

    def get_status_df(self) -> pd.DataFrame:
        """
        현재 보유 position 관련 정보를 반환하는 메서드

        Returns:
            pd.DataFrame: columns = [SYMBOL, CURRENT_QTY, CURRENT_PRICE, TRADE_PRICE]
        """
        current_symbol_list = list()
        _dict_df_result = self.dict_df_result
        _dict_df_position = self.dict_df_position

        _total_symbols = sorted(_dict_df_position.keys())

        for _symbol in _total_symbols:
            try:
                _symbol_result_df = _dict_df_result[_symbol]
                _symbol_position_df = _dict_df_position[_symbol]

                _current_price = (
                    _symbol_result_df.sort_values("DATE").tail(1)["PRICE"].values[0]
                )
                _trade_price = _symbol_position_df["TRADE_PRICE"].values[0]
                _current_qty = _symbol_position_df["QTY"].values[0]

                current_symbol_list.append(
                    {
                        "SYMBOL": _symbol,
                        "CURRENT_QTY": _current_qty,
                        "CURRENT_PRICE": _current_price,
                        "TRADE_PRICE": _trade_price,
                    }
                )
            except:
                pass
        return pd.DataFrame(
            current_symbol_list,
            columns=["SYMBOL", "CURRENT_QTY", "CURRENT_PRICE", "TRADE_PRICE"],
        )


class SYMBOL_LOADER:
    """
    SYMBOL_LOADER : 주식 symbol 정보 추출 클래스

    Inner_Classes :
        SYMBOl_FILTER

    Methods :
        - filter_symbols_df
        - get_symbols
        - __call__
    """

    @staticmethod
    def load_symbols_df() -> pd.DataFrame:
        """
        symbols_df를 호출하는 메서드

        Returns:
            pd.DataFrame :
        """
        symbols_df = kq.symbol_stock()
        return symbols_df

    class SYMBOL_FILTER:
        """
        SYMBOl_FILTER : 주식 symbol을 filtering 하는 클래스

        Methods :
                - filter__market
                - filter__admin_issue
                - filter__sec_type
        """

        @staticmethod
        def filter__market(symbols_df: pd.DataFrame) -> pd.DataFrame:
            """
            market에 대한 필터링을 진행하는 메서드

            Returns:
                pd.DataFrame : MARKET이 [코스닥, 유가증권]에 속하는 row만 유지
            """
            filtered_symbols_df = symbols_df[
                (symbols_df["MARKET"].isin(["코스닥", "유가증권"]))
            ].copy()
            return filtered_symbols_df

        @staticmethod
        def filter__admin_issue(symbols_df: pd.DataFrame) -> pd.DataFrame:
            """
            ADMIN_ISSUE에 대한 필터링을 진행하는 메서드

            Returns:
                pd.DataFrame : ADMIN_ISSUE가 0 인 row만 유지
            """
            filtered_symbols_df = symbols_df[(symbols_df["ADMIN_ISSUE"] == 0)].copy()
            return filtered_symbols_df

        @staticmethod
        def filter_sec_type(symbols_df: pd.DataFrame) -> pd.DataFrame:
            """
            SEC_TYPE에 대한 필터링을 진행하는 메서드

            Returns:
                pd.DataFrame : SEC_TYPE이 [ST, EF, EN]에 속하는 row만 유지
            """
            filtered_symbols_df = symbols_df[
                symbols_df["SEC_TYPE"].isin(["ST", "EF", "EN"])
            ].copy()
            return filtered_symbols_df

    def filter_symbols_df(self, symbols_df: pd.DataFrame) -> pd.DataFrame:
        """
        symbol_df 에 대한 필터링을 진행하는 메서드

        Returns:
                pd.DataFrame : SYMBOl_FILTER의 필터 메서드를 거친 row만 유지
        """
        symbol_filter = self.SYMBOL_FILTER()
        filtered_symbols_df = symbol_filter.filter__market(symbols_df)
        filtered_symbols_df = symbol_filter.filter__admin_issue(filtered_symbols_df)
        filtered_symbols_df = symbol_filter.filter_sec_type(filtered_symbols_df)
        return filtered_symbols_df

    @staticmethod
    def get_symbols(symbols_df: pd.DataFrame) -> list:
        """
        symbols_df의 symbol을 중복을 제거하여 추출하는 메서드

        Returns:
            list : symbols
        """
        symbols = sorted(set(symbols_df["SYMBOL"]))
        return symbols

    # SYMBOL_LOADER PIPELINE
    def __call__(self) -> list:
        """
        SYMBOL_LOADER의 파이프라인을 제공하는 메서드

        Returns:
            list : symbols
        """
        symbols_df = self.load_symbols_df()
        filtered_symbols_df = self.filter_symbols_df(symbols_df)
        symbols = self.get_symbols(filtered_symbols_df)
        return symbols

In [4]:
"""
Specific

DataLoader
    - FUNDAMENTAL_LOADER
    - PER_PROCESSOR
"""

class FUNDAMENTAL_LOADER:
    """
    FUNDAMENTAL_LOADER : symbol에 대한 fundamental_analysis를 위한 정보를 제공하는 클래스

    Methods :
        - __init__
        - load_recent_close
        - load_recent_marketcap
        - load_recent_netprofit
        - load_recent_capital
        - __call__
    """

    def __init__(self, symbol: str, date: dt.date) -> None:
        """
        FUNDAMENTAL_LOADER의 생성자

        Args:
            symbol (str):
            date (dt.date):

        Attr:
            symbol (str):
            date (dt.date):
            daily_stock_df (pd.DataFrame) :
                - kq.daily_stock 에서 호출
                - 현재 날짜부터 이전 7일까지 데이터
        """
        self.symbol = symbol
        self.date = date
        self.daily_stock_df = kq.daily_stock(
            symbol,
            start_date=date - dt.timedelta(days=7),
            end_date=date,
        )

    def load_recent_close(self) -> float:
        """
        가장 최근 종가를 가져온다.
        """
        daily_stock_df = self.daily_stock_df
        _close = daily_stock_df.sort_values("DATE").tail(1)["CLOSE"].values[0]
        return _close

    def load_recent_marketcap(self) -> float:
        """
        가장 최근 시가총액을 가져온다.
        """
        daily_stock_df = self.daily_stock_df
        _marketcap = daily_stock_df.sort_values("DATE").tail(1)["MARKETCAP"].values[0]
        return _marketcap

    def load_recent_netprofit(self) -> float:
        """
        공시자료 중 가장 최근 당기순이익을 가져온다.
        """
        netprofit_df = kq.account_history(self.symbol, "122700")
        netprofit_df.sort_values("YEARMONTH", inplace=True)
        _netprofit = netprofit_df.tail(1)["VALUE"].values[0]
        return _netprofit

    def load_recent_capital(self) -> float:
        """
        공시자료 중 가장 최근 총자산(총자산 - 총부채)를 가져온다.
        """
        capital_df = kq.account_history(self.symbol, "115000")
        capital_df.sort_values("YEARMONTH", inplace=True)
        _capital = capital_df.tail(1)["VALUE"].values[0]
        return _capital

    def __call__(self) -> dict:
        """
        fundmanetal analysis를 위해 필요한 데이터를 가져와서 dictionary를 반환한다.
        """
        _close = self.load_recent_close()
        _marketcap = self.load_recent_marketcap()
        _netprofit = self.load_recent_netprofit()
        _capital = self.load_recent_capital()
        return {
            "SYMBOL": self.symbol,
            "CLOSE": _close,
            "MARKETCAP": _marketcap,
            "NETPROFIT": _netprofit,
            "CAPITAL": _capital,
        }
        

import numpy as np


class PER_PROCESSOR:
    """
    FUDAMENTAL_PROCESSOR : 기본적 분석을 통한 order 생성 클래스

    Inner_Classes :
        - GET_BUYING_ORDERS
        - GET_SELLING_ORDERS

    Methods:
        - __init__

    """

    def __init__(self, fundamental_df: pd.DataFrame) -> None:
        """
        PER_PROCESSOR의 생성자

        Args:
            fundamental_df (pd.DataFrame)

        fundamental_df에 PER을 추가하고, PER이 음수인 row는 제거하여 저장합니다.
        """
        fundamental_df["PER"] = fundamental_df["MARKETCAP"] / (
            fundamental_df["NETPROFIT"]
        )
        fundamental_df = fundamental_df[fundamental_df["PER"] > 0]
        self.fundamental_df = fundamental_df

    class GET_BUYING_ORDERS:
        """
        GET_BUYING_ORDERS : 매수 주문을 생성하는 클래스

        Methods :
            - __init__
            - filter_position_symbols
            - get_low_per_df
            - append_per_weight
            - append_price_invest
            - append_cnt_invest
            - __call__
        """

        def __init__(
            self,
            fundamental_processor: FUNDAMENTAL_LOADER,
            daily_invest_money: float,
            position_symbols: list,
        ) -> None:
            """
            GET_BUYING_ORDERS의 생성자

            Args:
                - fundamental_processor (FUDAMENTAL_PROCESSOR의 인스턴스)
                - daily_invest_money (float)
                - position_symbols (list)
            """
            self.fundamental_processor = fundamental_processor
            self.daily_invest_money = daily_invest_money
            self.position_symbols = position_symbols

        @staticmethod
        def filter_position_symbols(
            fundamental_df: pd.DataFrame, position_symbols: list
        ) -> pd.DataFrame:
            """
            fundamental_df에서 이미 position이 있는 symbol들을 제거
                - 어느정도 분산 투자를 유지하기 위해서 입니다.

            Returns:
                pd.DataFrame : position_symbols이 filter된 fundamental_df
            """
            filtered_fundamental_df = fundamental_df[
                ~(fundamental_df["SYMBOL"].isin(position_symbols))
            ]
            return filtered_fundamental_df

        @staticmethod
        def get_low_per_df(fundamental_df: pd.DataFrame) -> pd.DataFrame:
            """
            fundamental_df에서 per이 낮은 5개를 선정하여 추출합니다.

            Returns:
                pd.DataFrame : fundamental_df의 per 낮은 5개 row 추출
            """
            low_per_df = fundamental_df.nsmallest(5, "PER")
            return low_per_df

        @staticmethod
        def append_per_weight(low_per_df: pd.DataFrame) -> pd.DataFrame:
            """
            low_per_df에서 per에 따른 투자금을 구하기 위해 weight를 구합니다.
                - per이 낮을 수록 비율이 증가합니다.

            Returns:
                pd.DataFrame : low_per_df에서 "per_WEIGHT" column 추가
            """
            low_per_df["PER_WEIGHT"] = low_per_df["PER"].sum() / low_per_df["PER"]
            return low_per_df

        @staticmethod
        def append_price_invest(
            low_per_df: pd.DataFrame, daily_invest_money: float
        ) -> pd.DataFrame:
            """
            low_per_df per_WEIGHT를 기준으로 투자금을 정합니다.
                - per_WEIGHT가 클수록 투자금이 커집니다.

            Returns:
                pd.DataFrame : low_per_df에서 "PRICE_INVEST" column 추가
            """
            low_per_df["PRICE_INVEST"] = (
                low_per_df["PER_WEIGHT"] / low_per_df["PER_WEIGHT"].sum()
            ) * daily_invest_money
            return low_per_df

        @staticmethod
        def append_cnt_invest(low_per_df: pd.DataFrame) -> pd.DataFrame:
            """
            low_per_df PRICE_INVEST를 기준으로 구매수량을 정합니다.

            Returns:
                pd.DataFrame : low_per_df에서 "CNT_INVEST" column 추가
            """
            low_per_df["CNT_INVEST"] = low_per_df["PRICE_INVEST"] // low_per_df["CLOSE"]
            return low_per_df

        def __call__(self) -> list:
            """
            GET_BUYING_ORDERS의 파이프라인을 제공하는 메서드

            - 추출 -
            1. fundamental_df 추출
            2. daily_invest_money 추출
            3. position_symbols 추출

            - 전처리 -
            1. position_symbols 필터링
            2. per_WEIGHT column 추가
            3. PRICE_INVEST column 추가
            4. CNT_INVEST column 추가

            - 결과 생성 -
            1. buying_orders 생성
                - [
                    ('symbol',count),
                    ('symbol',count),
                    ...
                ]

            Returns :
                buying_orders (list(tuple())) : 매수 주문
            """
            fundamental_df = self.fundamental_processor.fundamental_df
            daily_invest_money = self.daily_invest_money
            position_symbols = self.position_symbols

            filtered_fundamental_df = self.filter_position_symbols(
                fundamental_df, position_symbols
            )
            low_per_df = self.get_low_per_df(filtered_fundamental_df)
            low_per_df = self.append_per_weight(low_per_df)
            low_per_df = self.append_price_invest(low_per_df, daily_invest_money)
            low_per_df = self.append_cnt_invest(low_per_df)
            buying_orders = list(
                low_per_df.set_index("SYMBOL")["CNT_INVEST"]
                .astype(int)
                .to_dict()
                .items()
            )
            return buying_orders

    class GET_SELLING_ORDERS:
        """
        GET_SELLING_ORDERS : 매도 주문을 생성하는 클래스

        Methods :
            - __init__
            - get_limit_line
            - get_high_per_df
            - filter_position_symbols
            - __call__
        """

        def __init__(self, fundamental_processor, status_df) -> None:
            """
            GET_SELLING_ORDERS의 생성자

            Args:
                - fundamental_processor (FUDAMENTAL_PROCESSOR의 인스턴스)
                - status_df (pd.DataFrame)
            """
            self.fundamental_processor = fundamental_processor
            self.status_df = status_df

        @staticmethod
        def get_limit_line(fundamental_df: pd.DataFrame) -> float:
            """
            per 상위 75 % 를 한계선으로 설정하고 이를 기준값을 추출

            Returns :
                - limit_line (float)
            """
            limit_line = np.percentile(fundamental_df["PER"], 50)
            return limit_line

        @staticmethod
        def get_high_per_df(
            fundamental_df: pd.DataFrame, limit_line: float
        ) -> pd.DataFrame:
            """
            limit line을 기준으로 filtering 한 per 상위 fundamental_df 추출

            Returns :
                - high_per_df (pd.DataFrame) : 상위 per로 필터링된 fundamental_df
            """
            high_per_df = fundamental_df[fundamental_df["PER"] > limit_line]
            return high_per_df

        @staticmethod
        def filter_position_symbols(
            high_per_df: pd.DataFrame, position_symbols: list
        ) -> pd.DataFrame:
            """
            high_per_df중 position이 있는지 필터링

            Returns :
                - high_per_df (pd.DataFrame) : position으로 필터링 된 high_per_df 추출
            """
            filtered_position_symbols = sorted(
                set(high_per_df["SYMBOL"]) & set(position_symbols)
            )
            return filtered_position_symbols

        def __call__(self) -> list:
            """
            GET_SELLING_ORDERS의 파이프라인을 제공하는 메서드

            - 추출 -
            1. fundamental_df 추출
            2. status_df 추출
            3. position_symbols 추출

            - 전처리 -
            1. limit_line 추출
            2. high_per_df 생성
            3. high_per_df 필터링

            - 결과 생성 -
            1. selling_orders 생성
                - [
                    ('symbol',count),
                    ('symbol',count),
                    ...
                ]
            Returns :
                buying_orders (list(tuple())) : 매도 주문
            """
            fundamental_df = self.fundamental_processor.fundamental_df
            status_df = self.status_df
            position_symbols = sorted(set(status_df["SYMBOL"]))

            limit_line = self.get_limit_line(fundamental_df)
            high_per_df = self.get_high_per_df(fundamental_df, limit_line)
            filtered_position_symbols = self.filter_position_symbols(
                high_per_df, position_symbols
            )

            selling_df = status_df[status_df["SYMBOL"].isin(filtered_position_symbols)]

            selling_orders = list(
                selling_df.set_index("SYMBOL")["CURRENT_QTY"]
                .apply(lambda x: x * -1)
                .astype(int)
                .to_dict()
                .items()
            )
            return selling_orders

In [5]:
import random
import logging
import datetime as dt
import pandas as pd


def trade_func(
    date: dt.date,
    dict_df_result: dict[str, pd.DataFrame],
    dict_df_position: dict[str, pd.DataFrame],
    logger: logging.Logger,
) -> list[tuple[str, int]]:
    """
    STATUS_LOADER
        : get_current_cash()
            -> 현재 가용 가능한 현금을 가져옵니다.
        : get_status_df()
            -> 현재 포지션이 있는 주식들에 대한 정보를 가져옵니다.
    """
    status_loader = STATUS_LOADER(dict_df_result, dict_df_position)

    current_cash = status_loader.get_current_cash()
    daily_invest_money = current_cash / 2
    status_df = status_loader.get_status_df()
    position_symbols = sorted(set(status_df["SYMBOL"]))

    """
    SYMBOL_LOADER
        : __call__()
            -> 현재 시장에서 거래 가능한 symbol을 모두 가져옵니다.
    """
    symbol_loader = SYMBOL_LOADER()
    total_symbols = symbol_loader()

    sampled_symbols = random.sample(total_symbols, CFG["sampling_cnt"])
    using_symbols = sorted(set(sampled_symbols + position_symbols))
    """
    FUNDAMENTAL_LOADER
        : __call__()
            -> 특정 symbol에 대하여, fundamental anlysis를 위해 필요한 데이터를 추출합니다.
    """

    fundamental_data_list = list()
    for symbol in using_symbols:
        try:
            _fundamental_loader = FUNDAMENTAL_LOADER(symbol, date)
            _fundamental_data = _fundamental_loader()
            fundamental_data_list.append(_fundamental_data)
        except:
            pass
    fundamental_df = pd.DataFrame(fundamental_data_list)

    """
    PER_PROCESSOR
        : GET_BUYING_ORDERS
        : GET_SELLING_ORDERS
    """
    fundamental_processor = PER_PROCESSOR(fundamental_df)

    get_buying_orders = PER_PROCESSOR.GET_BUYING_ORDERS(
        fundamental_processor, daily_invest_money, position_symbols
    )
    buying_orders = get_buying_orders()
    get_selling_orders = PER_PROCESSOR.GET_SELLING_ORDERS(
        fundamental_processor, status_df
    )
    selling_orders = get_selling_orders()

    symbols_and_orders = buying_orders + selling_orders
    return symbols_and_orders

In [6]:
# loop
dict_df_result = kq.backtest_stock_port_daily(
    trade_func,
    "2023-08-01",  # 실제 심사에서는 투자기간 시작일
    "2023-08-31",  # 실제 심사에서는 투자기간 종료일
    init_cash=1_000_000_000,  # 10억원
)

[2023-08-01] 종목: 040610, 주문전 보유수량:      0 주문수량: 73,559, 매매수량: 73,559, 주문후 보유수량: 73,559
[2023-08-01] 종목: 317400, 주문전 보유수량:      0 주문수량: 22,746, 매매수량: 22,746, 주문후 보유수량: 22,746
[2023-08-01] 종목: 000850, 주문전 보유수량:      0 주문수량:  2,371, 매매수량:  2,371, 주문후 보유수량:  2,371
[2023-08-01] 종목: 297890, 주문전 보유수량:      0 주문수량: 14,862, 매매수량: 14,862, 주문후 보유수량: 14,862
[2023-08-01] 종목: 119850, 주문전 보유수량:      0 주문수량: 17,010, 매매수량: 17,010, 주문후 보유수량: 17,010
[2023-08-02] 종목: 106240, 주문전 보유수량:      0 주문수량: 90,915, 매매수량: 90,915, 주문후 보유수량: 90,915
[2023-08-02] 종목: 005010, 주문전 보유수량:      0 주문수량:  2,847, 매매수량:  2,847, 주문후 보유수량:  2,847
[2023-08-02] 종목: 036460, 주문전 보유수량:      0 주문수량:    656, 매매수량:    656, 주문후 보유수량:    656
[2023-08-02] 종목: 093050, 주문전 보유수량:      0 주문수량:    796, 매매수량:    796, 주문후 보유수량:    796
[2023-08-02] 종목: 084010, 주문전 보유수량:      0 주문수량:    844, 매매수량:    844, 주문후 보유수량:    844
[2023-08-03] 종목: 151860, 주문전 보유수량:      0 주문수량:  5,059, 매매수량:  5,059, 주문후 보유수량:  5,059
[2023-08-03] 종목: 039560, 주문전 보유수량:      0 주

In [7]:
dict_df_result['TOTAL']['TOTAL_VALUE'].tail(1)

21    983,461,720
Name: TOTAL_VALUE, dtype: int64